In [2]:
#Gradient Descent Function
def gradient_descent(X_train, y, lbd, alp, max_iter, costs):
  pheta = np.zeros(X_train.shape[1])
  cost = []

  for i in range(max_iter):
    j = np.dot(X_train, pheta)
    k = 1 / (1 + np.exp(-j))

    regu = lbd / y.size * pheta
    regu[0] = 0
    cst = (-y * np.log(k) - (1 - y) * np.log(1 - k)).mean()
    grad = np.dot(X_train.T, (k - y)) / y.size + regu
    pheta = pheta - alp * grad

    if i%100 == 0:
      cost.append(cst)
  
  return pheta, cost

#Predict Function
def predict(X_test, pheta):
  i = np.dot(X_test, pheta)
  ans = 1 / (1 + np.exp(-i))
  return ans

#Logistic Regression Function
def lr(X_train, X_test, y_train):
  train_inter = np.ones((X_train.shape[0], 1))
  test_inter = np.ones((X_test.shape[0], 1))
  X_train = np.concatenate((train_inter, X_train), axis = 1)
  X_test = np.concatenate((test_inter, X_test), axis = 1)
  i = set(y_train)
  j = []
  k = []

  lbd = 0
  alp = 0.2
  max_iter = 10000
  costs = False

  for l in i:
    y_temp = np.array(y_train == l, dtype = int)
    pheta, cost = gradient_descent(X_train, y_temp, lbd, alp, max_iter, costs)
    
    j.append(pheta)
    k.append(cost)

  #Probabilities
  test_predict = np.zeros((len(i), len(X_test)))
  for m in range(len(i)):
    test_predict[m,:] = predict(X_test, j[m]) 

  train_predict = np.zeros((len(i), len(X_train)))
  for n in range(len(i)):
    train_predict[n,:] = predict(X_train, y[n]) 

  #Max Probabilities
  y_test_predict = np.argmax(test_predict, axis = 0)
  y_train_predict = np.argmax(train_predict, axis = 0)

  #Solution
  results = {"Costs": cost, "y_test_predict": y_test_predict, "y_train_predict": y_train_predict, "learning_rate": alp, "max_iterations": max_iter, "lambda": lbd}

  return results


def kfoldCV(df, f):
  data = cross_validation_split(df, f)
  sol = []

  #Creating train and test sets
  for i in range(f):
    j = list(range(f))
    j.pop(i)
    for k in j:
      if k == j[0]:
        l = data[k]
      else:
        l = np.concatenate((l, data[k]), axis = 0)
    
    m = lr(l[:, 0:X_train.shape[1]], data[i][:, 0:X_train.shape[1]], l[:, X_train.shape[1]])
    n = m['y_predict']

    #Accuracy Calculation
    acc = (n == data[i][:, X_train.shape[1]]).sum()
    sol.append(acc / len(n))

  return sol


def cross_validation_split(df, f):
  split_data = []
  data = df
  f_shape = int(data.copy.shape[0] / f)

  #save folds
  for i in range(f):
    fold = []
    while len(fold) < f_shape:
      j = randrange(data.shape[0])
      k = data.index[j]
      fold.append(data.loc[k].values.tolist())
      data = data.drop(k)
    split_data.append(np.asarray(fold))

  return split_data

